# bert-large-cased-whole-word-masking-finetuned-squad μοντέλο και αυτόματο tokenization. Θεωρώ hallucinations τα datapoints στα οποία p(Hallucination) = 1. Threshold μέσω μέσου όρου των συνημιτόνων των γνωστών hallucinations. Προκύπτουν 32 πιθανά hallucinations εκ των οποίων το 46.875% είναι σωστό.

Θεωρώντας hallucinations τα datapoint με p(Hallucination) >= 0.66 προκύπτουν 29 πιθανά hallucinations απο τα οποία το 45.23% είναι σωστά

Για το αρχείο processed_df.json έχουμε 56.25% accuracy με 37 predicted hallucinations (p(Hallucination) = 1)

Για p(Hallucination) >= 0.66 47.61% accuracy με 33 predicted hallucinations

In [ ]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=17b46b30e4d79397c9994903331b0414ae02d0e3210caa700e56a1796d394f70
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import json
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

In [ ]:
model  = SentenceTransformer("bert-large-cased-whole-word-masking-finetuned-squad")

In [ ]:
with open("pg_processed_df.json", "r") as f:
    trial_data = json.load(f)


df = pd.json_normalize(trial_data)

#df.head(79)

In [ ]:
result_dict = {}
hallucination_cosines_total = 0
hallucinations_cosines = []
hallucinations = []
cosine_similarities = []

for i in range(0, len(df)):
    sentences = [df.iloc[i]['hyp'], df.iloc[i]['tgt']]
    embeddings = model.encode(sentences) #tokenization is implicitly handled by the SentenceTransformer here
    similarity = cosine_similarity(embeddings[0:1], embeddings[1:2])

    for id_h, h in enumerate(sentences):
        for id_t, t in enumerate(sentences):
            if id_h >= id_t:
                continue
            dp_id = len(result_dict)  #Using dp_id as the key
            #if df.iloc[i]['p(Hallucination)'] >= 0.66:
            if df.iloc[i]['p(Hallucination)'] == 1:
                hallucinations.append(dp_id)
                hallucination_cosines_total += similarity[0][0]
                hallucinations_cosines.append(similarity[0][0])

            result_dict[dp_id] = {'hyp': h, 'tgt': t, 'cosine_similarity': similarity[0][0]}


for dp_id, values in result_dict.items():
    cosine_similarities.append(values['cosine_similarity'])
    print(f'Datapoint {dp_id}:')
    print(f'hyp: {values["hyp"]}')
    print(f'tgt: {values["tgt"]}')
    print(f'cosine similarity: {values["cosine_similarity"]}')
    print()


Datapoint 0:
hyp: that’s happening moment
tgt: isn’t gonna happen
cosine similarity: 0.8340305685997009

Datapoint 1:
hyp: permit carry
tgt: uh i’m validated
cosine similarity: 0.795008659362793

Datapoint 2:
hyp: process easy
tgt: watch
cosine similarity: 0.77921062707901

Datapoint 3:
hyp: number five eight
tgt: 5 6 7 8
cosine similarity: 0.8790779113769531

Datapoint 4:
hyp: safer way travel
tgt: it’s safer
cosine similarity: 0.856874406337738

Datapoint 5:
hyp: you’re good scam artist
tgt: imposter
cosine similarity: 0.8198150396347046

Datapoint 6:
hyp: see
tgt: last time saw
cosine similarity: 0.7504163980484009

Datapoint 7:
hyp: let leave
tgt: someone let
cosine similarity: 0.7251225709915161

Datapoint 8:
hyp: impossible
tgt: there’s nothing can’t done
cosine similarity: 0.7050991058349609



In [ ]:
threshold = np.mean(hallucinations_cosines)

hallucinations_pred = []

# Iterate through result_dict and apply the thresholding logic
for dp_id, values in result_dict.items():
    similarity = values['cosine_similarity']
    if similarity < threshold:
        hallucinations_pred.append(dp_id)

counter1 = Counter(hallucinations)
counter2 = Counter(hallucinations_pred)

intersection = counter1 & counter2

total_mutual_elements = sum(intersection.values())

print(f'Threshold: {threshold}')
print(f'Total Hallucinations: {len(hallucinations)}')
print(f'Total Hallucinations Predicted: {len(hallucinations_pred)}')
print('Percentage of correctly predicted hallucinations:' , (total_mutual_elements / len(hallucinations)) * 100 , '%')
print('Correct Hallucinations:', intersection.keys())
print('Number of correct Hallucinations:', len(intersection.keys()))

Threshold: 0.8238994479179382
Total Hallucinations: 3
Total Hallucinations Predicted: 6
Percentage of correctly predicted hallucinations: 66.66666666666666 %
Correct Hallucinations: dict_keys([1, 5])
Number of correct Hallucinations: 2
